<a href="https://colab.research.google.com/github/xiaofanha/BigDataAnalytics/blob/master/YijieFangFlightsData5Questions.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

YijieFang's
Flights Data 5 Questions

In [1]:
!pip install --ignore-installed -q pyspark==3.0.1

     |████████████████████████████████| 204.2MB 74kB/s 
     |████████████████████████████████| 204kB 21.7MB/s 


In [2]:
import os
# os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-11-openjdk-amd64"
# os.environ["SPARK_HOME"] = "spark-3.0.1-bin-hadoop3.2"

# import findspark
# findspark.init("spark-3.0.1-bin-hadoop3.2")# SPARK_HOME


import sys
import requests
from operator import add

from pyspark import SparkConf,SparkContext
from pyspark.streaming import StreamingContext

from pyspark.sql import SparkSession
from pyspark.sql import SQLContext

from pyspark.sql.types import *
from pyspark.sql import functions as func
from pyspark.sql.functions import *

spark = SparkSession.builder.master("local[*]").getOrCreate()
sc = SparkContext.getOrCreate()
sqlContext = SQLContext(sc)

In [3]:
lines = sc.textFile("./flights.csv.bz2")

In [5]:
# Removing the Header Line from CSV file 
linesHeader = lines.first()
header = sc.parallelize([linesHeader])
linesWithOutHeader = lines.subtract(header)

In [6]:
flights = linesWithOutHeader.map(lambda x: x.split(','))
# YEAR,MONTH,DAY,DAY_OF_WEEK,AIRLINE,FLIGHT_NUMBER,TAIL_NUMBER,ORIGIN_AIRPORT,DESTINATION_AIRPORT,SCHEDULED_DEPARTURE,DEPARTURE_TIME,DEPARTURE_DELAY, CANCELLED
mainFlightsData = flights.map(lambda p: (p[0], p[1] , p[2] , p[3], p[4] , p[5] , p[6], p[7] , p[8] , p[9], p[10], p[11], p[24] ))

In [7]:
mainFlightsData.take(10)

[('2015',
  '1',
  '1',
  '4',
  'AA',
  '1028',
  'N3BFAA',
  'DEN',
  'ORD',
  '0600',
  '0601',
  '1',
  '0'),
 ('2015',
  '1',
  '1',
  '4',
  'WN',
  '828',
  'N8303R',
  'PHX',
  'BWI',
  '0610',
  '0610',
  '0',
  '0'),
 ('2015',
  '1',
  '1',
  '4',
  'AA',
  '1336',
  'N3LCAA',
  'PHX',
  'DFW',
  '0610',
  '0603',
  '-7',
  '0'),
 ('2015',
  '1',
  '1',
  '4',
  'NK',
  '231',
  'N620NK',
  'ATL',
  'FLL',
  '0700',
  '0725',
  '25',
  '0'),
 ('2015',
  '1',
  '1',
  '4',
  'UA',
  '1294',
  'N69830',
  'IAH',
  'MCO',
  '0727',
  '0727',
  '0',
  '0'),
 ('2015',
  '1',
  '1',
  '4',
  'US',
  '692',
  'N200UU',
  'CLT',
  'PHX',
  '0730',
  '0727',
  '-3',
  '0'),
 ('2015',
  '1',
  '1',
  '4',
  'MQ',
  '2952',
  'N698MQ',
  'GRB',
  'ORD',
  '0745',
  '0737',
  '-8',
  '0'),
 ('2015',
  '1',
  '1',
  '4',
  'DL',
  '1122',
  'N916DN',
  'ATL',
  'RSW',
  '0811',
  '0807',
  '-4',
  '0'),
 ('2015',
  '1',
  '1',
  '4',
  'WN',
  '3486',
  'N741SA',
  'BWI',
  'ABQ',
  '0825

In [21]:
#Q1 Get a list of  February departure delay time: (day,delay)pairs
#p[1]Month, p[2]Day, p[11]DEPARTURE_DELAY
FebDay_Delay = mainFlightsData.filter(lambda p: True if p[1] == "2" else False ).map(lambda p:(p[2],p[11]))
FebDay_Delay.take(10)

[('1', '-5'),
 ('1', '732'),
 ('1', '-7'),
 ('1', '15'),
 ('1', '-12'),
 ('1', '21'),
 ('1', ''),
 ('1', '-1'),
 ('1', '-4'),
 ('1', '-7')]

In [23]:
#Q2 Find the date(Year,Month,Day) which had the largest departure delay
#p[0]Year, p[1]Month, p[2]Day, p[11]DEPARTURE_DELAY
mainFlightsData.map(lambda p: (p[11], p[0], p[1], p[2])).top(1)
#The result is（2015,12,28）

[('999', '2015', '12', '28')]

In [26]:
#Q3 Which Month has the most cancellation of flights in 2015?
mainFlightsData_2015 = mainFlightsData.filter(lambda p: True if p[0] == "2015" else False)
Cancellation_2015 = mainFlightsData_2016.map(lambda p: (p[1],  1 ) if p[12] == "1" else (p[1],  0 )).reduceByKey(add)
Cancellation_2015.take(12)

[('1', 11982),
 ('2', 20517),
 ('9', 2075),
 ('8', 5052),
 ('10', 2454),
 ('5', 5694),
 ('6', 9120),
 ('7', 4806),
 ('4', 4520),
 ('3', 11002),
 ('11', 4599),
 ('12', 8063)]

In [27]:
Cancellation_2016.top(1, lambda x:x[1])
#Feb has the most cancellation of flights in 2015

[('2', 20517)]

In [30]:
#Q4 Which airport has the most cancellation of flights on weekend? 
#
weekEndFlights = flightsFromBoston.filter(lambda p: True if (int(p[3]) == 6 or int(p[3]) ==7) else False )
airportWithHighestCancellation = weekEndFlights.map(lambda p: (p[7],  1 ) if p[12] == "1" else (p[7],  0 )).reduceByKey(add).top(1, lambda x:x[1])

In [31]:
airportWithHighestCancellation

[('BOS', 476)]

In [33]:
#Q5 Which airline has the most flights? (from ORIGIN_AIRPORT to DESTINATION_AIRPORT)
#p[7]ORIGIN_AIRPORT, p[8]DESTINATION_AIRPORT
OG_DEST = mainFlightsData.map(lambda p: ((p[7],p[8]), 1)).reduceByKey(add)
OG_DEST.top(1, lambda x:x[1])
#From SFO to LAX has the most number of flights and the number of flights is 13744

[(('SFO', 'LAX'), 13744)]